In [1]:
#Compuerta analogica.
#Version de neuronas vectorizadas con numpy
import sys
import copy
import random
import numpy as np 
import math as math
import utils
import time
from neural_network import NeuralNetwork
from os import listdir
from PIL import Image as PImage
from PIL import ImageOps as ImageOps
import glob
import os

In [2]:
def parse_file(file):
 
    img = PImage.open(file)
    size= (64, 64)
    img.thumbnail(size)
    img = ImageOps.fit(img,size , PImage.ANTIALIAS)
    npImage=np.array(img, np.uint16)
    img_size = npImage.shape
    return npImage

In [3]:
def load_images_from_folder(folder, quantity):
    images = []
    for filename in os.listdir(folder)[:quantity]:        
        images.append(parse_file(folder+filename))
    return images

In [4]:
windows_path = 'C:/Users/carlos/OneDrive/machine_learning/nn/'

In [5]:
dataset_cats = load_images_from_folder(windows_path+'cat/',200)
testset_cats = np.array(dataset_cats)[101:200]
dataset_cats = np.array(dataset_cats)[:100]

print(dataset_cats[0].shape)

(64, 64, 3)


In [6]:
dataset_dogs = load_images_from_folder(windows_path+'dog/',200)
testset_dogs = np.array(dataset_dogs)[101:200]
dataset_dogs = np.array(dataset_dogs)[:100]

print(dataset_dogs.shape)

(100, 64, 64, 3)


In [7]:
samples_size = dataset_dogs.shape[0]
train_x_cats = (dataset_cats.reshape(samples_size, -1)) /255.
train_x_dogs = (dataset_dogs.reshape(samples_size, -1)) /255.
dataset_full = (np.concatenate((train_x_cats, train_x_dogs)))
train_x_full = dataset_full
#print(dataset_cats[0][0])
print(train_x_cats[0])

print(train_x_full.shape)

[0.95686275 0.78823529 0.47843137 ... 0.03137255 0.03137255 0.00392157]
(200, 12288)


In [8]:
samples_size = testset_dogs.shape[0]
test_x_cats = (testset_cats.reshape(samples_size, -1)) /255.
test_x_dogs = (testset_dogs.reshape(samples_size, -1)) /255.
testset_full = (np.concatenate((test_x_cats, test_x_dogs)))
test_x_full = testset_full
#print(dataset_cats[0][0])
print(test_x_cats[0])

print(test_x_full.shape)

[0.42352941 0.38431373 0.34509804 ... 0.71372549 0.6745098  0.63921569]
(198, 12288)


In [9]:
X = train_x_full
model_cat = np.array((1,0))
model_dog = np.array((0,1))


Y_cats = np.tile(model_cat,(train_x_cats.shape[0],1))
Y_dogs = np.tile(model_dog,(train_x_dogs.shape[0],1))



Y_full = np.array([np.concatenate((Y_cats, Y_dogs))])[0]
print(Y_full.shape)
nn = NeuralNetwork(X, Y_full, 32, 'tanh')
parameters = nn.get_parameters()
Y = Y_full


(200, 2)


In [10]:
print(X.shape) #(4, 2)
start_time = time.time()
info = nn.trainer(10000, 0.01, X.T, Y.T, log=False)
elapsed_time = time.time() - start_time
print('elapsed_time: ', elapsed_time)

(200, 12288)
elapsed_time:  174.86564135551453


In [11]:
Xt_true = train_x_cats
id_cat = 99
print(Xt_true[id_cat:id_cat+1].shape)
print(Xt_true[id_cat:id_cat+1])
t = nn.test(Xt_true[id_cat:id_cat+1].T)
print('t: ', t)
accy =  Y[id_cat:id_cat+1] /  t[0] 
print('accy: ',(accy))


(1, 12288)
[[0.90980392 0.91764706 0.90588235 ... 0.67058824 0.71372549 0.74117647]]
t:  [[0.97784303]
 [0.00837278]]
accy:  [[1.02265902 0.        ]]


In [12]:
Xt_false = train_x_dogs
id_dog = 99
print(Xt_false[id_dog:id_dog+1].shape)
print(Xt_false[id_dog:id_dog+1])
t = nn.test(Xt_false[id_dog:id_dog+1].T)
print('t: ', t)
accy =  Y[id_dog:id_dog+1] /  t[0] 
print('accy: ',(accy))

(1, 12288)
[[1.         1.         1.         ... 0.84313725 0.88235294 0.89019608]]
t:  [[-0.00644057]
 [ 0.99414902]]
accy:  [[-155.26572825   -0.        ]]


In [13]:
#Prueba data de entrenamiento
cats_count = 0
dogs_count = 0
for i in range(0,X.shape[0]):
    t = nn.test(X[i:i+1].T)
    if( t[0]>0.8):
        cats_count = cats_count +1
    if( t[1]>0.8):
        dogs_count = dogs_count +1

print('cats detected: ', cats_count)
print('dogs detected: ', dogs_count)

cats detected:  100
dogs detected:  100


In [14]:
#Prueba data de validacin 
cats_count = 0
dogs_count = 0
X_test = test_x_full
for i in range(0,X_test.shape[0]):
    t = nn.test(X[i:i+1].T)
    if( t[0]>0.8):
        cats_count = cats_count +1
    if( t[1]>0.8):
        dogs_count = dogs_count +1

print('cats validateds: ', cats_count)
print('dogs validateds: ', dogs_count)

cats validateds:  100
dogs validateds:  98


In [15]:
#probar coleccion que ya no existe en el dataset
image_test = load_images_from_folder(windows_path+'tests/',6)
testset_user = np.array(image_test)
samples_size = testset_user.shape[0]
test_flatten = (testset_user.reshape(samples_size, -1)) /255.


t = nn.test(test_flatten.T)
print('t: ', t)


t:  [[0.79706981 0.98164742 0.96114448 0.86600877 0.62877472 0.54530458]
 [0.74037418 0.12729427 0.50876045 0.77164298 0.91185414 0.94488404]]


In [23]:
#probar coleccion que ya no existe en el dataset
images = []
images.append(parse_file(windows_path+'/tests/dog.1000.jpg'))

image_test = images
testset_user = np.array(image_test)
samples_size = testset_user.shape[0]
test_flatten = (testset_user.reshape(samples_size, -1)) /255.


t = nn.test(test_flatten.T)
if t[0]>0.8:
    print('Its a cat!!')
if t[1]>0.8:
    print('Its a dog!!')
print('t: ', t)

Its a cat!!
t:  [[0.86600877]
 [0.77164298]]


In [17]:
model = np.array((1,0))
np.tile(model,(3,1))

array([[1, 0],
       [1, 0],
       [1, 0]])